# Imports

In [11]:
from sklearn.model_selection import train_test_split
import numpy as np
import matplotlib.pyplot as plt

In [19]:
import importlib
import data_loading
import data_processing

importlib.reload(data_loading)
importlib.reload(data_processing)

from data_loading import load_dataset, create_input_space, augment_data
from data_processing import preprocess_signals, normalize_data

In [13]:
data = load_dataset(signal_names=['ecg', 'gsr'])

In [14]:
print(len(data))
print(len(data[0]['signals']['ecg']))

8600
2816


# Preprocessing

In [15]:
data_filtered = preprocess_signals(data, 512, 256)

In [16]:
X, y = create_input_space(data_filtered)

(8600, 1408, 2)
(8600,)


In [20]:
X = normalize_data(X, local=False)

In [21]:
augmented_X, augmented_y = augment_data(X, y)

(77400, 1152, 2)
(77400,)


In [22]:
X_train, X_test, y_train, y_test = train_test_split(augmented_X, augmented_y, test_size=0.05, random_state=42)

# Model

In [1]:
import tensorflow as tf
from tensorflow.keras.layers import Input, Conv1D, MaxPooling1D, UpSampling1D, Add, GaussianNoise
from tensorflow.keras.models import Model

def create_encoder(input_shape, noise_level=0.1):
    """Create a denoising encoder model."""
    inputs = Input(shape=input_shape)
    x = GaussianNoise(noise_level)(inputs)
    x = Conv1D(32, 3, activation='relu', padding='same')(x)
    x = MaxPooling1D(2, padding='same')(x)
    x = Conv1D(16, 3, activation='relu', padding='same')(x)
    encoded = MaxPooling1D(2, padding='same')(x)
    return Model(inputs, encoded, name='encoder')

def create_decoder(encoded_shape):
    """Create a decoder model."""
    encoded_input = Input(shape=encoded_shape)
    x = Conv1D(16, 3, activation='relu', padding='same')(encoded_input)
    x = UpSampling1D(2)(x)
    x = Conv1D(32, 3, activation='relu', padding='same')(x)
    x = UpSampling1D(2)(x)
    decoded = Conv1D(1, 3, activation='sigmoid', padding='same')(x)
    return Model(encoded_input, decoded, name='decoder')

2023-12-12 15:28:05.611759: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 AVX_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [24]:
# Signal specific input shapes
signal_shapes = {
    'ECG': (1152, 1),
    'GSR': (1152, 1),
    # Add more signals as needed
}

# Creating a dictionary to hold each signal's autoencoder
autoencoders = {}

for signal_type, input_shape in signal_shapes.items():
    encoder = create_encoder(input_shape)
    decoder = create_decoder(encoder.output_shape[1:])

    autoencoder_input = Input(shape=input_shape)
    encoded = encoder(autoencoder_input)
    decoded = decoder(encoded)

    autoencoder = Model(autoencoder_input, decoded, name=f'autoencoder_{signal_type}')
    autoencoder.compile(optimizer='adam', loss='mean_squared_error')

    autoencoders[signal_type] = autoencoder
    autoencoder.summary()

Model: "autoencoder_ECG"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_3 (InputLayer)        [(None, 1152, 1)]         0         
                                                                 
 encoder (Functional)        (None, 288, 16)           1680      
                                                                 
 decoder (Functional)        (None, 1152, 1)           2449      
                                                                 
Total params: 4,129
Trainable params: 4,129
Non-trainable params: 0
_________________________________________________________________


2023-12-12 13:19:59.185971: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 AVX_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-12-12 13:19:59.188195: I tensorflow/core/common_runtime/process_util.cc:146] Creating new thread pool with default inter op setting: 2. Tune using inter_op_parallelism_threads for best performance.


Model: "autoencoder_GSR"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_6 (InputLayer)        [(None, 1152, 1)]         0         
                                                                 
 encoder (Functional)        (None, 288, 16)           1680      
                                                                 
 decoder (Functional)        (None, 1152, 1)           2449      
                                                                 
Total params: 4,129
Trainable params: 4,129
Non-trainable params: 0
_________________________________________________________________


In [1]:
import tensorflow as tf
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

2023-12-13 11:33:05.205888: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 AVX_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


Num GPUs Available:  0


In [26]:
# Assuming X_train and y_train are dictionaries with keys corresponding to signal types
# and values being the training data for each signal
X_train = {
    'ECG': X_train[:,:,0],  
    'GSR': X_train[:,:,1]
    # Add more signals as needed
}

# In case of autoencoders, usually, the target is the same as the input (for reconstruction tasks)
y_train = X_train

# Define the number of epochs and batch size for training
epochs = 50
batch_size = 32

# Training each autoencoder
for signal_type, autoencoder in autoencoders.items():
    print(f"Training autoencoder for {signal_type}...")
    autoencoder.fit(
        X_train[signal_type], y_train[signal_type],
        epochs=epochs,
        batch_size=batch_size,
        validation_split=0.2,  # Assuming you want to use 20% of the data for validation
        shuffle=True
    )


Training autoencoder for ECG...
Epoch 1/50
 361/1839 [====>.........................] - ETA: 2:28 - loss: 0.0118

KeyboardInterrupt: 